In [1]:
import pandas as pd
from pyspark.sql.functions import col, explode
from pyspark import SparkContext

In [2]:
from pyspark.sql import SparkSession
sc = SparkContext
# sc.setCheckpointDir('checkpoint')
spark = SparkSession.builder.appName('Recommendation').getOrCreate()

In [3]:
# membaca data
objekWisata = spark.read.csv("../dataset/cb_objekwisata.csv", header=True)
ratings = spark.read.csv("../dataset/data-rating-objek-wisata.csv", header=True)

In [4]:
ow = spark.read.csv("../dataset/cb_objekwisata.csv", header=True)
ow.show()

+--------+--------------------+--------------------+
|id_objek|          nama_objek|          type_objek|
+--------+--------------------+--------------------+
|     999|  AEON Mall BSD City|      TEMPAT BELANJA|
|     998|Agrowisata Teba S...| WISATA ALAM & TAMAN|
|     997|    Air Panas Banjar| WISATA ALAM & TAMAN|
|     996|Air Terjun Aling-...| WISATA ALAM & TAMAN|
|     995|Air Terjun Benang...| WISATA ALAM & TAMAN|
|     994|Air Terjun Coban ...| WISATA ALAM & TAMAN|
|     993|Air Terjun Cunca ...| WISATA ALAM & TAMAN|
|     992|Air Terjun Kanto ...| WISATA ALAM & TAMAN|
|     991|Air Terjun Madaka...| WISATA ALAM & TAMAN|
|     990| Air Terjun Nungnung| WISATA ALAM & TAMAN|
|     989|Air Terjun Sendan...| WISATA ALAM & TAMAN|
|     988|Air Terjun Sipiso...| WISATA ALAM & TAMAN|
|     987|Air Terjun Sri Ge...| WISATA ALAM & TAMAN|
|     986|Air Terjun Tegenu...| WISATA ALAM & TAMAN|
|     985|Air Terjun Tukad ...| WISATA ALAM & TAMAN|
|     984|Airport Railink S...|        TRANSPO

In [5]:
objekWisata.show()

+--------+--------------------+--------------------+
|id_objek|          nama_objek|          type_objek|
+--------+--------------------+--------------------+
|     999|  AEON Mall BSD City|      TEMPAT BELANJA|
|     998|Agrowisata Teba S...| WISATA ALAM & TAMAN|
|     997|    Air Panas Banjar| WISATA ALAM & TAMAN|
|     996|Air Terjun Aling-...| WISATA ALAM & TAMAN|
|     995|Air Terjun Benang...| WISATA ALAM & TAMAN|
|     994|Air Terjun Coban ...| WISATA ALAM & TAMAN|
|     993|Air Terjun Cunca ...| WISATA ALAM & TAMAN|
|     992|Air Terjun Kanto ...| WISATA ALAM & TAMAN|
|     991|Air Terjun Madaka...| WISATA ALAM & TAMAN|
|     990| Air Terjun Nungnung| WISATA ALAM & TAMAN|
|     989|Air Terjun Sendan...| WISATA ALAM & TAMAN|
|     988|Air Terjun Sipiso...| WISATA ALAM & TAMAN|
|     987|Air Terjun Sri Ge...| WISATA ALAM & TAMAN|
|     986|Air Terjun Tegenu...| WISATA ALAM & TAMAN|
|     985|Air Terjun Tukad ...| WISATA ALAM & TAMAN|
|     984|Airport Railink S...|        TRANSPO

In [6]:
ratings.show()

+-------+--------------------+--------------------+------+--------+--------------------+--------+
|id_user|                name|            username|rating|    date|          nama_objek|id_objek|
+-------+--------------------+--------------------+------+--------+--------------------+--------+
|   4363|        Compass73198|/Profile/Compass7...|     1|  19-Sep|Agrowisata Teba S...|     998|
|   6237|              Emc921|     /Profile/Emc921|     1|Okt 2019|Agrowisata Teba S...|     998|
|   7446|           gaddavino|  /Profile/gaddavino|     1|Agt 2019|Agrowisata Teba S...|     998|
|  10391|                 K&C| /Profile/963kristyb|     1|Okt 2019|Agrowisata Teba S...|     998|
|  12803|           Michael H|/Profile/MichaelH...|     1|  19-Jul|Agrowisata Teba S...|     998|
|  16435|                Rona|/Profile/Ronaindo...|     1|  19-Jul|Agrowisata Teba S...|     998|
|  11566|           Lindsay H|/Profile/986lindsayh|     1|  19-Jan|    Air Panas Banjar|     997|
|  11788|           

In [7]:
ratings.printSchema()

root
 |-- id_user: string (nullable = true)
 |-- name: string (nullable = true)
 |-- username: string (nullable = true)
 |-- rating: string (nullable = true)
 |-- date: string (nullable = true)
 |-- nama_objek: string (nullable = true)
 |-- id_objek: string (nullable = true)



In [8]:
ratings = ratings.\
    withColumn('id_user', col('id_user').cast('integer')).\
    withColumn('id_objek', col('id_objek').cast('integer')).\
    withColumn('rating', col('rating').cast('integer'))
ratings.show()

+-------+--------------------+--------------------+------+--------+--------------------+--------+
|id_user|                name|            username|rating|    date|          nama_objek|id_objek|
+-------+--------------------+--------------------+------+--------+--------------------+--------+
|   4363|        Compass73198|/Profile/Compass7...|     1|  19-Sep|Agrowisata Teba S...|     998|
|   6237|              Emc921|     /Profile/Emc921|     1|Okt 2019|Agrowisata Teba S...|     998|
|   7446|           gaddavino|  /Profile/gaddavino|     1|Agt 2019|Agrowisata Teba S...|     998|
|  10391|                 K&C| /Profile/963kristyb|     1|Okt 2019|Agrowisata Teba S...|     998|
|  12803|           Michael H|/Profile/MichaelH...|     1|  19-Jul|Agrowisata Teba S...|     998|
|  16435|                Rona|/Profile/Ronaindo...|     1|  19-Jul|Agrowisata Teba S...|     998|
|  11566|           Lindsay H|/Profile/986lindsayh|     1|  19-Jan|    Air Panas Banjar|     997|
|  11788|           

In [9]:
# groupby data objek wisata by type objek dan memunculkan id baru untuk setiap tipe objek
from pyspark.sql.functions import row_number
from pyspark.sql.window import Window
w = Window.orderBy("count")
objekWisata_type = objekWisata.groupBy("type_objek").count().select("*").withColumn("id_type_objek", row_number().over(w)).drop('count')
objekWisata_type.show()

+--------------------+-------------+
|          type_objek|id_type_objek|
+--------------------+-------------+
|KONSER & PERTUNJUKAN|            1|
|SUMBER INFORMASI ...|            2|
|        TRANSPORTASI|            3|
|KEBUN BINATANG & ...|            4|
|             LAINNYA|            5|
|TAMAN AIR & TAMAN...|            6|
|              MUSEUM|            7|
|      TEMPAT BELANJA|            8|
|PEMANDANGAN & TEN...|            9|
| WISATA ALAM & TAMAN|           10|
+--------------------+-------------+



In [10]:
# grouping data rating berdasarkan type objek, dengan nilai max rating dan menggabungkan dengan data tipe objek wisata
from pyspark.sql.functions import max
ratings_type = ratings.join(objekWisata, on="nama_objek").groupBy(["type_objek","id_user","name","username"]).agg(max("rating"))
ratings_type = ratings_type.join(objekWisata_type, on="type_objek")
ratings_type.show()

+--------------------+-------+------------------+--------------------+-----------+-------------+
|          type_objek|id_user|              name|            username|max(rating)|id_type_objek|
+--------------------+-------+------------------+--------------------+-----------+-------------+
|              MUSEUM|  19407|         Valerie B|/Profile/valerieb567|          4|            7|
|             LAINNYA|  16406|          Roland V|/Profile/rolandva...|          1|            5|
|PEMANDANGAN & TEN...|   1369|      alleajourney|/Profile/alleajou...|          5|            9|
|      TEMPAT BELANJA|   8643|              Hula|  /Profile/hulapiggy|          2|            8|
|      TEMPAT BELANJA|  10871|    KirsifromTurku|/Profile/Kirsifro...|          2|            8|
|             LAINNYA|  16459|             Roo T|    /Profile/RoannaS|          2|            5|
|      TEMPAT BELANJA|  17590|          siralex1|   /Profile/siralex1|          3|            8|
|PEMANDANGAN & TEN...|  19736|

In [11]:
# menampilkan komposisi sparse date untuk rating objek
numerator = ratings.select("rating").count()

num_users = ratings.select("id_user").distinct().count()
num_objekWisata =  ratings.select("id_objek").distinct().count()

denominator = num_users * num_objekWisata

sparsity = (1.0 - (numerator *1.0)/denominator)*100
print("The ratings dataframe is ", "%.2f" % sparsity + "% empty.")

The ratings dataframe is  99.52% empty.


In [12]:
# menampilkan komposisi sparse date untuk rating type objek
numerator_type = ratings_type.select("max(rating)").count()

num_users_type = ratings_type.select("id_user").distinct().count()
num_objekWisata_type =  ratings_type.select("id_type_objek").distinct().count()

denominator_type = num_users_type * num_objekWisata_type

sparsity_type = (1.0 - (numerator_type *1.0)/denominator_type)*100
print("The ratings dataframe is ", "%.2f" % sparsity_type + "% empty.")

The ratings dataframe is  85.52% empty.


In [13]:
# Group data by userId, count ratings sbg input model rating objek, dan menampilkan jumlah rating per user
username_ratings = ratings.groupBy("id_user").count().orderBy('count', ascending=False)
username_ratings.show()

+-------+-----+
|id_user|count|
+-------+-----+
|   2108|  105|
|  18575|   73|
|   2509|   72|
|  18895|   62|
|   9515|   55|
|   8424|   53|
|   8158|   50|
|   4567|   49|
|  11169|   49|
|   1282|   46|
|   8574|   46|
|   5303|   42|
|   9177|   42|
|   6338|   41|
|   9242|   39|
|   3228|   38|
|  16265|   38|
|  20858|   38|
|   7910|   38|
|  18052|   36|
+-------+-----+
only showing top 20 rows



In [14]:
# Group data by userId, count ratings sbg input model rating tipe objek, dan menampilkan jumlah rating per user
username_ratings_type = ratings_type.groupBy("id_user").count().orderBy('count', ascending=False)
username_ratings_type.show()

+-------+-----+
|id_user|count|
+-------+-----+
|  13297|   14|
|  16265|   14|
|   5799|   12|
|   5303|   11|
|  20062|    9|
|   4567|    9|
|  14243|    9|
|   8449|    9|
|   4739|    9|
|   9177|    9|
|   8509|    8|
|   5789|    8|
|   8424|    8|
|  14107|    8|
|  13705|    8|
|   1218|    8|
|  12011|    8|
|  13329|    8|
|   8574|    8|
|  18575|    8|
+-------+-----+
only showing top 20 rows



In [15]:
# jumlah rating per objek
attraction_ratings = ratings.groupBy("id_objek").count().orderBy('count', ascending=False)
attraction_ratings.show()

+--------+-----+
|id_objek|count|
+--------+-----+
|     833|  100|
|     858|  100|
|     623|  100|
|     897|  100|
|     737|  100|
|     808|  100|
|     580|  100|
|     883|  100|
|     879|  100|
|     898|  100|
|     804|  100|
|     588|  100|
|     970|  100|
|     799|  100|
|     853|  100|
|     974|  100|
|     961|  100|
|     857|  100|
|     633|  100|
|     673|  100|
+--------+-----+
only showing top 20 rows



In [16]:
# jumlah rating per tipe objek
attraction_ratings_type = ratings_type.groupBy("id_type_objek").count().orderBy('count', ascending=False)
attraction_ratings_type.show()

+-------------+-----+
|id_type_objek|count|
+-------------+-----+
|           10|11551|
|            9| 6785|
|            8| 4370|
|            7| 2460|
|            6| 2086|
|            4|  841|
|            5|  838|
|            3|  393|
|            2|  292|
|            1|  197|
+-------------+-----+



In [17]:
# Import fungsi rekomendasi ALS dari pyspark
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator

In [18]:
# set train & test dataset
(train, test) = ratings.randomSplit([0.8, 0.2], seed = 1234)

# konfugarasi ALS model
als = ALS(userCol="id_user", itemCol="id_objek", ratingCol="rating", nonnegative = True, implicitPrefs = False, coldStartStrategy="drop")

# menampilkan model ALS, konfirmasi
type(als)

pyspark.ml.recommendation.ALS

In [19]:
# set train & test dataset
(t_train, t_test) = ratings_type.randomSplit([0.8, 0.2], seed = 1234)

# konfugarasi ALS model
t_als = ALS(userCol="id_user", itemCol="id_type_objek", ratingCol="max(rating)", nonnegative = True, implicitPrefs = False, coldStartStrategy="drop")

# menampilkan model ALS, konfirmasi
type(t_als)

pyspark.ml.recommendation.ALS

In [20]:
# konfigurasi parameter untuk hyper parameter tuning
param_grid = ParamGridBuilder() \
            .addGrid(als.rank, [10, 50, 100, 150]) \
            .addGrid(als.regParam, [.01, .05, .1, .15]) \
            .build()
            #             .addGrid(als.maxIter, [5, 50, 100, 200]) \

           
# konfigurasi evaluator, menggunakan RMSE
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating", predictionCol="prediction") 
evaluator_t = RegressionEvaluator(metricName="rmse", labelCol="max(rating)", predictionCol="prediction") 
print ("Num models to be tested: ", len(param_grid))

Num models to be tested:  16


In [21]:
# Bmembuat cross validation
cv = CrossValidator(estimator=als, estimatorParamMaps=param_grid, evaluator=evaluator, numFolds=5)
t_cv = CrossValidator(estimator=t_als, estimatorParamMaps=param_grid, evaluator=evaluator_t, numFolds=5)

# konfirmasi fungsi cross validation sudah dibuat
print(cv)
print(t_cv)

CrossValidator_6b43ac2fbb6b
CrossValidator_38d22113bc36


In [22]:
# training model
model = cv.fit(train)               # untuk rating objek
t_model = t_cv.fit(t_train)         # untuk rating tipe objek

# ekstraksi best model
best_model = model.bestModel
best_t_model = t_model.bestModel

In [23]:
# Print best_model
print(type(best_model))

# Complete the code below to extract the ALS model parameters
print("**Best Model**")

# # Print "Rank"
print("  Rank:", best_model._java_obj.parent().getRank())

# Print "MaxIter"
print("  MaxIter:", best_model._java_obj.parent().getMaxIter())

# Print "RegParam"
print("  RegParam:", best_model._java_obj.parent().getRegParam())

<class 'pyspark.ml.recommendation.ALSModel'>
**Best Model**
  Rank: 150
  MaxIter: 10
  RegParam: 0.05


In [24]:
# Print best_model
print(type(best_t_model))

# Complete the code below to extract the ALS model parameters
print("**Best Model**")

# # Print "Rank"
print("  Rank:", best_t_model._java_obj.parent().getRank())

# Print "MaxIter"
print("  MaxIter:", best_t_model._java_obj.parent().getMaxIter())

# Print "RegParam"
print("  RegParam:", best_t_model._java_obj.parent().getRegParam())

<class 'pyspark.ml.recommendation.ALSModel'>
**Best Model**
  Rank: 10
  MaxIter: 10
  RegParam: 0.1


In [25]:
# menampilkan prediksi & RMSE untuk model rating objek
test_predictions = best_model.transform(test)
RMSE = evaluator.evaluate(test_predictions)
print(RMSE)

0.804418366942342


In [26]:
# menampilkan prediksi & RMSE untuk model rating tipe objek
test_predictions_t = best_t_model.transform(t_test)
RMSE_t = evaluator_t.evaluate(test_predictions_t)
print(RMSE_t)

1.2269424261795425


In [27]:
test_predictions.show()

+-------+---------------+--------------------+------+--------+--------------------+--------+----------+
|id_user|           name|            username|rating|    date|          nama_objek|id_objek|prediction|
+-------+---------------+--------------------+------+--------+--------------------+--------+----------+
|   8130|     Hardi Prya|  /Profile/239hardip|     4|  18-Jan|         Kuta Square|     833|   4.03539|
|  18738|        ths23id|    /Profile/ths23id|     3|  16-Jul|         Kuta Square|     833| 2.9974778|
|    417|   588chiquitap|/Profile/588chiqu...|     4|  16-Jul|         Kuta Square|     833| 3.9652443|
|   2967|  Bagus Perbawa| /Profile/bawa167799|     5|  19-Sep|         Kuta Square|     833| 4.2277484|
|   7559|      Genaddi P|/Profile/genaddib...|     3|Okt 2017|         Kuta Square|     833| 3.9455504|
|  18361|            T S|   /Profile/ts200777|     3|  17-Nov|         Kuta Square|     833| 2.9989498|
|   8652|        HUSIN G| /Profile/husing2017|     4|  18-Jun|  

In [28]:
test_predictions_t.show()

+--------------------+-------+--------------------+--------------------+-----------+-------------+----------+
|          type_objek|id_user|                name|            username|max(rating)|id_type_objek|prediction|
+--------------------+-------+--------------------+--------------------+-----------+-------------+----------+
|KONSER & PERTUNJUKAN|  12366|       MarvinSitorus|/Profile/MarvinSi...|          4|            1|   3.09079|
|KONSER & PERTUNJUKAN|   2924|                 B G|  /Profile/245bevang|          5|            1|  2.631522|
|KONSER & PERTUNJUKAN|  19936|           wforwiddi| /Profile/widdimouse|          4|            1| 2.0369678|
|KONSER & PERTUNJUKAN|    332|           2014FpvGt|  /Profile/2014FpvGt|          2|            1| 3.4860935|
|KONSER & PERTUNJUKAN|  10124|          Jonathan S|/Profile/P891HYjo...|          5|            1| 3.6878774|
|KONSER & PERTUNJUKAN|  15948|             Richard| /Profile/312eugenep|          5|            1| 3.4368434|
|KONSER & 

In [29]:
# membuat rekomendasi untuk semua user berdasarkan model rating objek
nrecommendations = best_model.recommendForAllUsers(10)
nrecommendations.limit(20).show()

+-------+--------------------+
|id_user|     recommendations|
+-------+--------------------+
|   4900|[{898, 5.501377},...|
|   5300|[{905, 5.1349773}...|
|   6620|[{862, 5.0761924}...|
|   7240|[{573, 4.981666},...|
|   7340|[{863, 4.978387},...|
|   7880|[{915, 4.915547},...|
|   9900|[{940, 1.1015375}...|
|  12940|[{901, 5.23668}, ...|
|  13840|[{898, 4.958282},...|
|  14450|[{679, 4.2652454}...|
|  14570|[{679, 5.1900115}...|
|  15790|[{873, 4.979742},...|
|  17420|[{898, 5.0704393}...|
|  18800|[{803, 4.928554},...|
|  19530|[{940, 5.5905666}...|
|    471|[{940, 3.103029},...|
|   1591|[{621, 4.034756},...|
|   4101|[{953, 4.978831},...|
|  11141|[{688, 4.8876643}...|
|  16861|[{772, 2.9897919}...|
+-------+--------------------+



In [30]:
# membuat rekomendasi untuk semua user berdasarkan model rating tipe objek
nrecommendations_t = best_t_model.recommendForAllUsers(10)
nrecommendations_t.limit(20).show()

+-------+--------------------+
|id_user|     recommendations|
+-------+--------------------+
|   1580|[{10, 4.815191}, ...|
|   4900|[{9, 4.8058424}, ...|
|   5300|[{8, 4.808525}, {...|
|   6620|[{8, 4.798581}, {...|
|   7240|[{6, 4.7999806}, ...|
|   7340|[{10, 4.815191}, ...|
|   7880|[{6, 1.9199922}, ...|
|   9900|[{8, 0.961705}, {...|
|  12940|[{9, 4.7493668}, ...|
|  13840|[{10, 4.815191}, ...|
|  14450|[{3, 3.8457851}, ...|
|  14570|[{8, 4.808525}, {...|
|  15790|[{6, 4.7999806}, ...|
|  17420|[{9, 4.8186035}, ...|
|  18800|[{9, 4.7067823}, ...|
|  19530|[{9, 4.8010607}, ...|
|    471|[{10, 2.8891144},...|
|   1591|[{9, 3.854883}, {...|
|   4101|[{9, 4.8186035}, ...|
|  11141|[{8, 4.4072037}, ...|
+-------+--------------------+



In [31]:
nrecommendations = nrecommendations\
    .withColumn("rec_ex", explode("recommendations"))\
    .select('id_user', col("rec_ex.id_objek"), col("rec_ex.rating"))

nrecommendations.limit(10).show()

+-------+--------+---------+
|id_user|id_objek|   rating|
+-------+--------+---------+
|   4900|     898| 5.501377|
|   4900|     940|5.4956956|
|   4900|     679| 5.478775|
|   4900|     620|5.4573293|
|   4900|     621| 5.457173|
|   4900|     933|5.4551897|
|   4900|     862|5.4172025|
|   4900|     681|5.4163394|
|   4900|     905|5.4096556|
|   4900|     680| 5.407803|
+-------+--------+---------+



In [32]:
# rekomendasi untuk spesifik user id, model rating objek
nrecommendations.join(objekWisata, on='id_objek').filter('id_user = 15').show()

+--------+-------+------+----------+----------+
|id_objek|id_user|rating|nama_objek|type_objek|
+--------+-------+------+----------+----------+
+--------+-------+------+----------+----------+



In [33]:
nrecommendations_t = nrecommendations_t\
    .withColumn("rec_ex", explode("recommendations"))\
    .select('id_user', col("rec_ex.id_type_objek"), col("rec_ex.rating"))

nrecommendations.limit(10).show()

+-------+--------+---------+
|id_user|id_objek|   rating|
+-------+--------+---------+
|   4900|     898| 5.501377|
|   4900|     940|5.4956956|
|   4900|     679| 5.478775|
|   4900|     620|5.4573293|
|   4900|     621| 5.457173|
|   4900|     933|5.4551897|
|   4900|     862|5.4172025|
|   4900|     681|5.4163394|
|   4900|     905|5.4096556|
|   4900|     680| 5.407803|
+-------+--------+---------+



In [34]:
# rekomendasi untuk spesifik user id, model rating type objek
nrecommendations_t.join(objekWisata_type, on='id_type_objek').filter('id_user = 15').show()

+-------------+-------+------+----------+
|id_type_objek|id_user|rating|type_objek|
+-------------+-------+------+----------+
+-------------+-------+------+----------+



In [35]:
# fungsi rekomendasi berdasarkan user id
def recommendation(uid):
    df1 = nrecommendations.join(objekWisata, on='id_objek')
    df1 = df1.selectExpr("id_objek", "id_user", "rating as rating_objek", "nama_objek", "type_objek")
    df2 = nrecommendations_t.join(objekWisata_type, on='id_type_objek')
    df2 = df2.selectExpr("id_type_objek", "id_user", "rating as rating_type", "type_objek")
    
    # menggabungkan rekomendasi rating objek & rating type objek
    df = df1.join(df2, ['id_user','type_objek']).filter('id_user = ' + str(uid))

    # menggabung rekomendasi skor antara rating objek dan rating tipe objek
    df = df.withColumn('rating', (df['rating_objek']*df['rating_type']) ** (1/2))
    df = df.drop('rating_objek','rating_type','id_type_objek')
    print('Rekomendasi Objek Pariwisata untuk user dengan id: ', uid)
    return df.show()

In [36]:
# rekomendasi untuk random user
from random import randint
n = randint(0,len(ratings.toPandas()['id_user'].unique()))
random_user = ratings.toPandas()['id_user'].unique()[n]
recommendation(100)

Rekomendasi Objek Pariwisata untuk user dengan id:  100
+-------+----------+--------+----------+------+
|id_user|type_objek|id_objek|nama_objek|rating|
+-------+----------+--------+----------+------+
+-------+----------+--------+----------+------+



In [37]:
len(ratings.toPandas()['id_user'].unique())

20722

In [53]:
ratings.toPandas()['id_user'].unique()[890]

13663

In [64]:
recommendation(17854)

Rekomendasi Objek Pariwisata untuk user dengan id:  17854
+-------+-------------------+--------+--------------------+------------------+
|id_user|         type_objek|id_objek|          nama_objek|            rating|
+-------+-------------------+--------+--------------------+------------------+
|  17854|WISATA ALAM & TAMAN|     940|Danau Kelimutu (D...|3.3556774993728706|
|  17854|WISATA ALAM & TAMAN|     679|      Pulau Pahawang|3.3489660062252127|
|  17854|WISATA ALAM & TAMAN|     621|Taman Nasional Br...| 3.347768062829688|
|  17854|WISATA ALAM & TAMAN|     620|Taman Nasional Gu...|3.3427621683548727|
|  17854|WISATA ALAM & TAMAN|     898|      Gunung Rinjani|3.3402015182097884|
|  17854|WISATA ALAM & TAMAN|     862|          Kawah Ijen| 3.337371875146673|
|  17854|WISATA ALAM & TAMAN|     680|         Pulau Padar|3.3304605661533717|
|  17854|WISATA ALAM & TAMAN|     905|   Gunung Api Semeru|3.3208448275020572|
|  17854|     TEMPAT BELANJA|     809|Matahari Kuta Square|3.912813699352

In [65]:
recommendation(688)

Rekomendasi Objek Pariwisata untuk user dengan id:  688
+-------+--------------------+--------+--------------------+-----------------+
|id_user|          type_objek|id_objek|          nama_objek|           rating|
+-------+--------------------+--------+--------------------+-----------------+
|    688| WISATA ALAM & TAMAN|     898|      Gunung Rinjani|4.759621562820373|
|    688| WISATA ALAM & TAMAN|     679|      Pulau Pahawang|4.743926130198928|
|    688| WISATA ALAM & TAMAN|     621|Taman Nasional Br...|4.724330002850148|
|    688| WISATA ALAM & TAMAN|     862|          Kawah Ijen|4.716740060152133|
|    688| WISATA ALAM & TAMAN|     620|Taman Nasional Gu...|4.710600630575813|
|    688| WISATA ALAM & TAMAN|     905|   Gunung Api Semeru|4.710294511986319|
|    688| WISATA ALAM & TAMAN|     940|Danau Kelimutu (D...|4.701256872089438|
|    688| WISATA ALAM & TAMAN|     745|Pantai Kuta - Lombok|4.690842110973573|
|    688|PEMANDANGAN & TEN...|     846|   Keraton Kasepuhan|4.3395380690190

In [62]:
recommendation(455)

Rekomendasi Objek Pariwisata untuk user dengan id:  455
+-------+--------------------+--------+--------------------+------------------+
|id_user|          type_objek|id_objek|          nama_objek|            rating|
+-------+--------------------+--------+--------------------+------------------+
|    455|PEMANDANGAN & TEN...|     806|     Mesjid Al-Akbar|3.4248020421200933|
|    455| WISATA ALAM & TAMAN|     606|        Tanjung Bira| 3.916240033119686|
|    455| WISATA ALAM & TAMAN|     679|      Pulau Pahawang|3.8975678660789113|
|    455| WISATA ALAM & TAMAN|     898|      Gunung Rinjani|3.8891987465226836|
|    455| WISATA ALAM & TAMAN|     621|Taman Nasional Br...| 3.888687325355419|
|    455| WISATA ALAM & TAMAN|     940|Danau Kelimutu (D...|3.8877090496020803|
|    455| WISATA ALAM & TAMAN|     620|Taman Nasional Gu...|3.8830595801353422|
|    455| WISATA ALAM & TAMAN|     862|          Kawah Ijen| 3.869825415209536|
|    455| WISATA ALAM & TAMAN|     745|Pantai Kuta - Lombok|3.86

In [60]:
recommendation(265)

Rekomendasi Objek Pariwisata untuk user dengan id:  265
+-------+--------------------+--------+--------------------+------------------+
|id_user|          type_objek|id_objek|          nama_objek|            rating|
+-------+--------------------+--------+--------------------+------------------+
|    265| WISATA ALAM & TAMAN|     679|      Pulau Pahawang| 4.498686916781354|
|    265| WISATA ALAM & TAMAN|     680|         Pulau Padar| 4.468111219300835|
|    265| WISATA ALAM & TAMAN|     940|Danau Kelimutu (D...| 4.453749013981481|
|    265| WISATA ALAM & TAMAN|     618|Taman Nasional Lo...| 4.400351397614212|
|    265| WISATA ALAM & TAMAN|     862|          Kawah Ijen|4.3924710882773645|
|    265| WISATA ALAM & TAMAN|     621|Taman Nasional Br...| 4.390033722837045|
|    265| WISATA ALAM & TAMAN|     620|Taman Nasional Gu...| 4.387871907439714|
|    265| WISATA ALAM & TAMAN|     898|      Gunung Rinjani| 4.374642493763598|
|    265| WISATA ALAM & TAMAN|     905|   Gunung Api Semeru| 4.3

In [59]:
recommendation(1250)

Rekomendasi Objek Pariwisata untuk user dengan id:  1250
+-------+--------------------+--------+--------------------+------------------+
|id_user|          type_objek|id_objek|          nama_objek|            rating|
+-------+--------------------+--------+--------------------+------------------+
|   1250| WISATA ALAM & TAMAN|     940|Danau Kelimutu (D...|5.0614406277894535|
|   1250| WISATA ALAM & TAMAN|     679|      Pulau Pahawang| 5.012175232219209|
|   1250| WISATA ALAM & TAMAN|     680|         Pulau Padar|  5.00776793747236|
|   1250| WISATA ALAM & TAMAN|     621|Taman Nasional Br...| 4.990586001554929|
|   1250| WISATA ALAM & TAMAN|     620|Taman Nasional Gu...| 4.987374195625139|
|   1250| WISATA ALAM & TAMAN|     681|     Pulau Menjangan| 4.962701534916058|
|   1250| WISATA ALAM & TAMAN|     862|          Kawah Ijen| 4.950181537728665|
|   1250| WISATA ALAM & TAMAN|     901|     Gunung Kelimutu| 4.940054896431895|
|   1250|              MUSEUM|     878|Jatiluwih Warisan...| 4.

In [58]:
recommendation(800)

Rekomendasi Objek Pariwisata untuk user dengan id:  800
+-------+-------------------+--------+--------------------+------------------+
|id_user|         type_objek|id_objek|          nama_objek|            rating|
+-------+-------------------+--------+--------------------+------------------+
|    800|            LAINNYA|     973|        Bali Theater|0.8314502818967435|
|    800|WISATA ALAM & TAMAN|     679|      Pulau Pahawang|0.8830192121357578|
|    800|WISATA ALAM & TAMAN|     898|      Gunung Rinjani|0.8790870650053098|
|    800|WISATA ALAM & TAMAN|     621|Taman Nasional Br...|0.8757562094383066|
|    800|WISATA ALAM & TAMAN|     620|Taman Nasional Gu...|0.8738119711339872|
|    800|WISATA ALAM & TAMAN|     680|         Pulau Padar|0.8735342692098277|
|    800|WISATA ALAM & TAMAN|     862|          Kawah Ijen|0.8704718178575365|
|    800|WISATA ALAM & TAMAN|     940|Danau Kelimutu (D...|0.8698361132262377|
|    800|WISATA ALAM & TAMAN|     745|Pantai Kuta - Lombok|0.86790021184546

In [57]:
recommendation(221)

Rekomendasi Objek Pariwisata untuk user dengan id:  221
+-------+--------------------+--------+--------------------+------------------+
|id_user|          type_objek|id_objek|          nama_objek|            rating|
+-------+--------------------+--------+--------------------+------------------+
|    221| WISATA ALAM & TAMAN|     940|Danau Kelimutu (D...|3.5108095547256934|
|    221| WISATA ALAM & TAMAN|     679|      Pulau Pahawang|3.5057849759254083|
|    221| WISATA ALAM & TAMAN|     862|          Kawah Ijen| 3.490018234861114|
|    221| WISATA ALAM & TAMAN|     680|         Pulau Padar| 3.475607888821519|
|    221| WISATA ALAM & TAMAN|     621|Taman Nasional Br...| 3.470539490594211|
|    221| WISATA ALAM & TAMAN|     620|Taman Nasional Gu...|3.4646027662902474|
|    221| WISATA ALAM & TAMAN|     898|      Gunung Rinjani| 3.454880212233673|
|    221| WISATA ALAM & TAMAN|     905|   Gunung Api Semeru| 3.452672177827125|
|    221| WISATA ALAM & TAMAN|     681|     Pulau Menjangan|3.44

In [56]:
recommendation(13663)

Rekomendasi Objek Pariwisata untuk user dengan id:  13663
+-------+--------------------+--------+--------------------+------------------+
|id_user|          type_objek|id_objek|          nama_objek|            rating|
+-------+--------------------+--------+--------------------+------------------+
|  13663| WISATA ALAM & TAMAN|     940|Danau Kelimutu (D...|2.3783673958677123|
|  13663| WISATA ALAM & TAMAN|     679|      Pulau Pahawang|2.3566836739681354|
|  13663| WISATA ALAM & TAMAN|     680|         Pulau Padar| 2.352459302463036|
|  13663| WISATA ALAM & TAMAN|     621|Taman Nasional Br...|2.3160450542791535|
|  13663| WISATA ALAM & TAMAN|     620|Taman Nasional Gu...|2.3136960622311196|
|  13663| WISATA ALAM & TAMAN|     862|          Kawah Ijen| 2.312998331348891|
|  13663| WISATA ALAM & TAMAN|     681|     Pulau Menjangan|2.3079942676863068|
|  13663| WISATA ALAM & TAMAN|     747|   Pantai Kelingking|2.3045723579222175|
|  13663|              MUSEUM|     878|Jatiluwih Warisan...|2.

In [54]:
recommendation(890)

Rekomendasi Objek Pariwisata untuk user dengan id:  890
+-------+-------------------+--------+--------------------+------------------+
|id_user|         type_objek|id_objek|          nama_objek|            rating|
+-------+-------------------+--------+--------------------+------------------+
|    890|             MUSEUM|     878|Jatiluwih Warisan...| 3.563900321003223|
|    890|     TEMPAT BELANJA|     653|Samasta Lifestyle...|3.4867019068272462|
|    890|WISATA ALAM & TAMAN|     621|Taman Nasional Br...| 4.122538901678141|
|    890|WISATA ALAM & TAMAN|     898|      Gunung Rinjani|  4.11341062163002|
|    890|WISATA ALAM & TAMAN|     620|Taman Nasional Gu...|  4.11195878290975|
|    890|WISATA ALAM & TAMAN|     679|      Pulau Pahawang| 4.103958563522941|
|    890|WISATA ALAM & TAMAN|     905|   Gunung Api Semeru| 4.093397052488469|
|    890|WISATA ALAM & TAMAN|     940|Danau Kelimutu (D...| 4.092322646709982|
|    890|WISATA ALAM & TAMAN|     862|          Kawah Ijen| 4.0906010521405

In [52]:
recommendation(7259)

Rekomendasi Objek Pariwisata untuk user dengan id:  7259
+-------+--------------------+--------+--------------------+------------------+
|id_user|          type_objek|id_objek|          nama_objek|            rating|
+-------+--------------------+--------+--------------------+------------------+
|   7259|PEMANDANGAN & TEN...|     806|     Mesjid Al-Akbar| 1.052815110431808|
|   7259|PEMANDANGAN & TEN...|     814|     Masjid Istiqlal|1.0483496798305738|
|   7259| WISATA ALAM & TAMAN|     745|Pantai Kuta - Lombok|1.0378534840868205|
|   7259| WISATA ALAM & TAMAN|     747|   Pantai Kelingking| 1.035482391226122|
|   7259| WISATA ALAM & TAMAN|     679|      Pulau Pahawang|1.0328768988541885|
|   7259| WISATA ALAM & TAMAN|     940|Danau Kelimutu (D...|1.0310216130600682|
|   7259| WISATA ALAM & TAMAN|     905|   Gunung Api Semeru|1.0307588852807752|
|   7259| WISATA ALAM & TAMAN|     624|Taman Laut Nasion...|1.0306111871407924|
|   7259| WISATA ALAM & TAMAN|     621|Taman Nasional Br...|1.0

In [39]:
recommendation(985)

Rekomendasi Objek Pariwisata untuk user dengan id:  985
+-------+--------------------+--------+--------------------+------------------+
|id_user|          type_objek|id_objek|          nama_objek|            rating|
+-------+--------------------+--------+--------------------+------------------+
|    985|PEMANDANGAN & TEN...|     933|       Desa Wae Rebo| 2.016876778284122|
|    985| WISATA ALAM & TAMAN|     679|      Pulau Pahawang|1.7736160927807694|
|    985| WISATA ALAM & TAMAN|     940|Danau Kelimutu (D...|1.7735048524947739|
|    985| WISATA ALAM & TAMAN|     862|          Kawah Ijen|1.7605972949777606|
|    985| WISATA ALAM & TAMAN|     898|      Gunung Rinjani|1.7603055779733923|
|    985| WISATA ALAM & TAMAN|     621|Taman Nasional Br...|1.7554984225594983|
|    985| WISATA ALAM & TAMAN|     681|     Pulau Menjangan|1.7524535826278358|
|    985| WISATA ALAM & TAMAN|     620|Taman Nasional Gu...|1.7522770508833265|
|    985| WISATA ALAM & TAMAN|     680|         Pulau Padar|1.75

In [40]:
recommendation(1231)

Rekomendasi Objek Pariwisata untuk user dengan id:  1231
+-------+--------------------+--------+--------------------+------------------+
|id_user|          type_objek|id_objek|          nama_objek|            rating|
+-------+--------------------+--------+--------------------+------------------+
|   1231|PEMANDANGAN & TEN...|     933|       Desa Wae Rebo|1.7625574116115152|
|   1231| WISATA ALAM & TAMAN|     940|Danau Kelimutu (D...|2.0011523026703055|
|   1231| WISATA ALAM & TAMAN|     621|Taman Nasional Br...|  1.99970439872153|
|   1231| WISATA ALAM & TAMAN|     620|Taman Nasional Gu...|1.9988438721752737|
|   1231| WISATA ALAM & TAMAN|     679|      Pulau Pahawang|1.9958697228774358|
|   1231| WISATA ALAM & TAMAN|     681|     Pulau Menjangan| 1.989888439118467|
|   1231| WISATA ALAM & TAMAN|     862|          Kawah Ijen|1.9894414787917818|
|   1231| WISATA ALAM & TAMAN|     905|   Gunung Api Semeru|1.9887519084506466|
|   1231| WISATA ALAM & TAMAN|     745|Pantai Kuta - Lombok| 1.

In [41]:
recommendation(2648)

Rekomendasi Objek Pariwisata untuk user dengan id:  2648
+-------+--------------------+--------+--------------------+------------------+
|id_user|          type_objek|id_objek|          nama_objek|            rating|
+-------+--------------------+--------+--------------------+------------------+
|   2648|TAMAN AIR & TAMAN...|     615|Taman Rekreasi Ja...| 2.730114303962641|
|   2648| WISATA ALAM & TAMAN|     765|   Pantai Batu Belig|3.7390248227801606|
|   2648| WISATA ALAM & TAMAN|     747|   Pantai Kelingking| 3.499830241854847|
|   2648| WISATA ALAM & TAMAN|     745|Pantai Kuta - Lombok|3.4822524923315257|
|   2648| WISATA ALAM & TAMAN|     898|      Gunung Rinjani|3.4805578012772935|
|   2648| WISATA ALAM & TAMAN|     658|Pusat Konservasi ...|3.4326076024492775|
|   2648| WISATA ALAM & TAMAN|     621|Taman Nasional Br...|3.4240068039781937|
|   2648| WISATA ALAM & TAMAN|     940|Danau Kelimutu (D...|3.4233147366274608|
|   2648| WISATA ALAM & TAMAN|     620|Taman Nasional Gu...|3.4

In [42]:
recommendation(19969)

Rekomendasi Objek Pariwisata untuk user dengan id:  19969
+-------+--------------------+--------+--------------------+------------------+
|id_user|          type_objek|id_objek|          nama_objek|            rating|
+-------+--------------------+--------+--------------------+------------------+
|  19969|PEMANDANGAN & TEN...|     571|       Yellow Bridge| 4.896646223828408|
|  19969|PEMANDANGAN & TEN...|     604|Tebing Blue Lagoo...| 4.552632151093896|
|  19969| WISATA ALAM & TAMAN|     600|        Telaga Warna| 4.057569122947877|
|  19969| WISATA ALAM & TAMAN|     602|Tebing Devil`s Tears| 4.021929708374024|
|  19969| WISATA ALAM & TAMAN|     940|Danau Kelimutu (D...|3.9573799725257923|
|  19969| WISATA ALAM & TAMAN|     747|   Pantai Kelingking|3.9523858747708807|
|  19969| WISATA ALAM & TAMAN|     745|Pantai Kuta - Lombok|3.9515517651346874|
|  19969| WISATA ALAM & TAMAN|     898|      Gunung Rinjani|3.9502327331354867|
|  19969| WISATA ALAM & TAMAN|     621|Taman Nasional Br...| 3

In [43]:
recommendation(579)

Rekomendasi Objek Pariwisata untuk user dengan id:  579
+-------+--------------------+--------+--------------------+------------------+
|id_user|          type_objek|id_objek|          nama_objek|            rating|
+-------+--------------------+--------+--------------------+------------------+
|    579| WISATA ALAM & TAMAN|     965|Beji Guwang Hidde...|3.8741189508801672|
|    579| WISATA ALAM & TAMAN|     679|      Pulau Pahawang|2.9313969622049507|
|    579| WISATA ALAM & TAMAN|     680|         Pulau Padar| 2.857418370586966|
|    579| WISATA ALAM & TAMAN|     734|     Pantai Nusa Dua| 2.852876543338881|
|    579| WISATA ALAM & TAMAN|     602|Tebing Devil`s Tears|2.8034607481000506|
|    579| WISATA ALAM & TAMAN|     986|Air Terjun Tegenu...|2.8029727209739175|
|    579|PEMANDANGAN & TEN...|     664|      Pura Saraswati| 2.766014976808461|
|    579|PEMANDANGAN & TEN...|     641|St. Francis Xavie...| 2.506555259930977|
|    579|PEMANDANGAN & TEN...|     958| Campuhan Ridge Walk|2.50

In [44]:
recommendation(641)

Rekomendasi Objek Pariwisata untuk user dengan id:  641
+-------+--------------------+--------+--------------------+------------------+
|id_user|          type_objek|id_objek|          nama_objek|            rating|
+-------+--------------------+--------+--------------------+------------------+
|    641|PEMANDANGAN & TEN...|     933|       Desa Wae Rebo|0.8855621592343119|
|    641|              MUSEUM|     878|Jatiluwih Warisan...|0.8598265068686298|
|    641| WISATA ALAM & TAMAN|     679|      Pulau Pahawang|1.0006414978459086|
|    641| WISATA ALAM & TAMAN|     940|Danau Kelimutu (D...|0.9991525695215666|
|    641| WISATA ALAM & TAMAN|     680|         Pulau Padar|0.9952696738618667|
|    641| WISATA ALAM & TAMAN|     681|     Pulau Menjangan|0.9877456600911647|
|    641| WISATA ALAM & TAMAN|     747|   Pantai Kelingking|0.9857191868822103|
|    641| WISATA ALAM & TAMAN|     745|Pantai Kuta - Lombok|0.9845117141003481|
|    641| WISATA ALAM & TAMAN|     898|      Gunung Rinjani|0.98

In [45]:
recommendation(10408)

Rekomendasi Objek Pariwisata untuk user dengan id:  10408
+-------+--------------------+--------+--------------------+------------------+
|id_user|          type_objek|id_objek|          nama_objek|            rating|
+-------+--------------------+--------+--------------------+------------------+
|  10408|             LAINNYA|     973|        Bali Theater|0.7585902990857881|
|  10408| WISATA ALAM & TAMAN|     996|Air Terjun Aling-...|0.9795640517824726|
|  10408| WISATA ALAM & TAMAN|     745|Pantai Kuta - Lombok|0.8981570262149703|
|  10408| WISATA ALAM & TAMAN|     747|   Pantai Kelingking|0.8978467906719713|
|  10408| WISATA ALAM & TAMAN|     679|      Pulau Pahawang|0.8943561232605405|
|  10408| WISATA ALAM & TAMAN|     862|          Kawah Ijen|0.8931577374042282|
|  10408| WISATA ALAM & TAMAN|     621|Taman Nasional Br...|0.8859049217518005|
|  10408| WISATA ALAM & TAMAN|     940|Danau Kelimutu (D...|0.8855627649982483|
|  10408|PEMANDANGAN & TEN...|     933|       Desa Wae Rebo| 0

In [46]:
recommendation(10043)

Rekomendasi Objek Pariwisata untuk user dengan id:  10043
+-------+--------------------+--------+--------------------+-----------------+
|id_user|          type_objek|id_objek|          nama_objek|           rating|
+-------+--------------------+--------+--------------------+-----------------+
|  10043| WISATA ALAM & TAMAN|     997|    Air Panas Banjar|4.685288988643549|
|  10043| WISATA ALAM & TAMAN|     996|Air Terjun Aling-...|4.562901727417792|
|  10043| WISATA ALAM & TAMAN|     965|Beji Guwang Hidde...|4.344107853261874|
|  10043| WISATA ALAM & TAMAN|     745|Pantai Kuta - Lombok| 4.23549533113581|
|  10043|             LAINNYA|     973|        Bali Theater|3.874363631631634|
|  10043|PEMANDANGAN & TEN...|     933|       Desa Wae Rebo|3.626278092894536|
|  10043|PEMANDANGAN & TEN...|     627|      Taman Festival|3.609568801234736|
|  10043|PEMANDANGAN & TEN...|     664|      Pura Saraswati|3.606295903011244|
|  10043|PEMANDANGAN & TEN...|     641|St. Francis Xavie...| 3.6004282060

In [47]:
recommendation(6203)

Rekomendasi Objek Pariwisata untuk user dengan id:  6203
+-------+--------------------+--------+--------------------+------------------+
|id_user|          type_objek|id_objek|          nama_objek|            rating|
+-------+--------------------+--------+--------------------+------------------+
|   6203| WISATA ALAM & TAMAN|     658|Pusat Konservasi ...| 4.593551112430313|
|   6203| WISATA ALAM & TAMAN|     709|Pantai Tanjung Pe...|  4.20096575214943|
|   6203| WISATA ALAM & TAMAN|     747|   Pantai Kelingking| 4.178556085914699|
|   6203| WISATA ALAM & TAMAN|     745|Pantai Kuta - Lombok|4.1466557183892885|
|   6203| WISATA ALAM & TAMAN|     974|Bali Sea Turtle S...| 4.142479132333246|
|   6203| WISATA ALAM & TAMAN|     759|   Pantai Double Six| 4.142301169799613|
|   6203| WISATA ALAM & TAMAN|     739|        Pantai Mawun| 4.131495130598625|
|   6203|      TEMPAT BELANJA|     653|Samasta Lifestyle...|3.3536946377217314|
|   6203|              MUSEUM|     878|Jatiluwih Warisan...| 3.

In [48]:
recommendation(5859)

Rekomendasi Objek Pariwisata untuk user dengan id:  5859
+-------+--------------------+--------+--------------------+------------------+
|id_user|          type_objek|id_objek|          nama_objek|            rating|
+-------+--------------------+--------+--------------------+------------------+
|   5859|PEMANDANGAN & TEN...|     933|       Desa Wae Rebo|1.8303246591589741|
|   5859|PEMANDANGAN & TEN...|     806|     Mesjid Al-Akbar|1.8073792303531686|
|   5859| WISATA ALAM & TAMAN|     940|Danau Kelimutu (D...|2.0788856204276955|
|   5859| WISATA ALAM & TAMAN|     862|          Kawah Ijen|2.0750342308805827|
|   5859| WISATA ALAM & TAMAN|     679|      Pulau Pahawang| 2.065013625351703|
|   5859| WISATA ALAM & TAMAN|     680|         Pulau Padar| 2.053474561408791|
|   5859| WISATA ALAM & TAMAN|     621|Taman Nasional Br...|2.0521613427323944|
|   5859| WISATA ALAM & TAMAN|     620|Taman Nasional Gu...|2.0509770181857174|
|   5859| WISATA ALAM & TAMAN|     898|      Gunung Rinjani| 2.